In [3]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

def get_youtube_shorts_titles_and_views(url):
    # Set up the Selenium webdriver
    driver = webdriver.Chrome()  # Ensure the correct ChromeDriver version is being used
    driver.get(url)

    # Wait for the page to load
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

    ## Use the code below to get titles and Views from Shorts instaed of Longform Videos

    '''
    # Click the "Shorts" tab using the correct XPath
    try:
        shorts_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//yt-formatted-string[@title='Shorts' and text()='Shorts']"))
        )
        shorts_button.click()
    except Exception as e:
        print(f"Error: {e}")
        driver.quit()
        return []   
    '''   
    
    # Scroll down to load more videos
    SCROLL_PAUSE_TIME = 2
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

        # Wait to load the page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:  # If we've reached the bottom, stop scrolling
            break
        last_height = new_height

    # Get the page source and parse it with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Extract the shorts video titles and views
    titles_and_views = []
    for result in soup.find_all('a', id='video-title'):
        title = result.text.strip()

        # Extract the views from the next sibling span with the specified class
        views_tag = result.find_next('span', class_='inline-metadata-item style-scope ytd-video-meta-block')
        views = views_tag.text.strip() if views_tag else "N/A"  # If views are found, store them

        titles_and_views.append((title, views))  # Append a tuple of (title, views)

    # Close the webdriver
    driver.quit()

    return titles_and_views

# Function to save titles and views to CSV
def save_titles_and_views_to_csv(titles_and_views, filename):
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['No.', 'Title', 'Views'])  # Write the header
        for i, (title, views) in enumerate(titles_and_views[:10], start=1):  # Set the number of titles that you want to collect
            writer.writerow([i, title, views])  # Write each title and views in the CSV

# URL to scrape
url = "https://www.youtube.com/results?search_query=data+analysis+with+excel" #Post your URL here to get the desired results
titles_and_views = get_youtube_shorts_titles_and_views(url)

# Save the titles and views to a CSV file
csv_filename = 'Test Scrapping V2.csv' # Rename the file as you want
save_titles_and_views_to_csv(titles_and_views, csv_filename)

print(f"Successfully saved {len(titles_and_views)} titles and views to '{csv_filename}'")

Successfully saved 612 titles and views to 'Test Scrapping V2.csv'
